#### PRNG evolution analysis with prediction attacks

In [1]:
#Basic imports for now

# NN-Tools
import numpy as np 
import keras 

# Visualization
from IPython.display import SVG 
from IPython.display import display
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

# Printing
from sympy import *
init_printing(use_latex=True)

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
from PRNGs import *
from SeedGenerator import *
import time

t = ticks()

print(t)
start_time = time.time()
ls = Lagged_Fibonacci(int(t),10000)
print("--- %s seconds ---" % (time.time() - start_time))

print(ls)


6.372148955270568e+17
Lagged_Fibonacci


IndexError: string index out of range

In [18]:
j = 3
k = 7
s = [8, 6, 7, 5, 3, 0, 9]
ls = []
sub = []
for z in range(10000):
    for n in range(len(s)):
        for i in range(len(s)):
            if i is 0:
                out = (s[j-1] + s[k-1]) % 10 # the pseudorandom output
                #print(out)
                sub.append(out)
            elif 0 < i < 6:
                s[i] = s[i+1] # shift the array
            else:
                s[i] = out
    ls.append(sub)
    s = sub
    sub = []
    
    
    
print(ls)
        

[[6, 8, 1, 0, 0, 4, 2], [4, 3, 3, 7, 9, 2, 5], [3, 5, 4, 6, 1, 9, 4], [8, 4, 5, 4, 8, 6, 0], [8, 9, 7, 3, 3, 8, 7], [5, 7, 0, 8, 5, 9, 6], [4, 4, 9, 8, 4, 0, 4], [6, 1, 5, 5, 9, 2, 3], [3, 3, 2, 4, 7, 5, 8], [8, 4, 1, 6, 4, 4, 8], [0, 5, 9, 3, 1, 0, 5], [9, 7, 8, 8, 3, 7, 4], [4, 0, 3, 0, 4, 6, 6], [2, 9, 3, 9, 5, 4, 3], [9, 5, 0, 4, 7, 3, 8], [6, 2, 9, 2, 0, 8, 0], [8, 1, 1, 9, 9, 8, 9], [9, 9, 8, 6, 5, 5, 4], [0, 8, 3, 8, 2, 4, 2], [2, 3, 5, 9, 1, 6, 9], [5, 3, 4, 0, 9, 3, 6], [4, 0, 9, 2, 8, 8, 8], [0, 9, 7, 5, 3, 0, 9], [7, 1, 4, 4, 3, 9, 0], [6, 8, 1, 0, 0, 4, 2], [4, 3, 3, 7, 9, 2, 5], [3, 5, 4, 6, 1, 9, 4], [8, 4, 5, 4, 8, 6, 0], [8, 9, 7, 3, 3, 8, 7], [5, 7, 0, 8, 5, 9, 6], [4, 4, 9, 8, 4, 0, 4], [6, 1, 5, 5, 9, 2, 3], [3, 3, 2, 4, 7, 5, 8], [8, 4, 1, 6, 4, 4, 8], [0, 5, 9, 3, 1, 0, 5], [9, 7, 8, 8, 3, 7, 4], [4, 0, 3, 0, 4, 6, 6], [2, 9, 3, 9, 5, 4, 3], [9, 5, 0, 4, 7, 3, 8], [6, 2, 9, 2, 0, 8, 0], [8, 1, 1, 9, 9, 8, 9], [9, 9, 8, 6, 5, 5, 4], [0, 8, 3, 8, 2, 4, 2], [2, 3, 5, 

In [ ]:
#initially using fewer kernels and increasing gradually while monitoring the error rate on how it is varying
model = keras.Sequential()
model.add(keras.layers.Conv1D(64, kernel_size=2,
    activation='relu',
    input_shape=()) 
        
    #data work in progress
          
model.add(keras.layers.Conv1D(64, 2, activation='relu'))
model.add(keras.layers.Conv1D(64, 2, activation='relu'))
model.add(keras.layers.Conv1D(64, 2, activation='relu'))

#test pool size and monitor what works better

model.add(keras.layers.MaxPooling1D(pool_size=(4)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(4, activation='relu'))
model.add(keras.layers.Dropout(0.5))
          
          
model.add(keras.layers.Dense(1, activation='softmax'))
          
    #data work in progress
          
model.compile(loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'])
          
model.summary()